# Rolling Planning mit Flow Time

In [2]:
import time

In [3]:
import pandas as pd
pd.set_option('display.max_rows', 20)

In [4]:
import utils.presenter as show
import utils.checker as check

import utils.scheduling_solver as ssv

from ProductionDaySimulation import ProductionDaySimulation

In [5]:
import utils.rolling_planning.init_jobs_times as rp_init
import utils.rolling_planning.procedure as rp_proced

## Generierung des Datensatzes für 10 Tage (Ankunft)

In [7]:
df_instance = pd.read_csv("data/00_instance.csv")

df_jssp, df_times = rp_init.init_jobs_with_arrivals(df_instance, 10,  u_b_mmax = 0.95)
df_jssp

,Job,Operation,Machine,Processing Time
0,Job_000,0,M0,29
1,Job_000,1,M1,78
2,Job_000,2,M2,9
3,Job_000,3,M3,36
4,Job_000,4,M4,49
...,...,...,...,...
1895,Job_189,5,M9,76
1896,Job_189,6,M5,47
1897,Job_189,7,M3,52
1898,Job_189,8,M4,90


In [8]:
df_times

,Job,Arrival
0,Job_000,0.00
1,Job_001,83.42
2,Job_002,144.74
3,Job_003,197.03
4,Job_004,233.63
...,...,...
185,Job_185,14088.21
186,Job_186,14094.27
187,Job_187,14095.63
188,Job_188,14214.35


## Params

In [31]:
max_time = 60*60*4 # 4h

In [10]:
# fix
day_length = 1440
horizon_days = 3

In [11]:
# init
day_numb = 0

day_start = 0
day_end = 0
planning_end = 0

# Initialisierung (Tag 0)

In [13]:
day_numb = 0

In [14]:
day_start = day_length*day_numb                      # 0
day_end = day_start + day_length                     # 1440
planning_end =  day_start + horizon_days*day_length  # 4320

In [15]:
# I)
df_jssp_curr, df_times_curr = rp_proced.filter_jobs_by_arrival_window(df_times, df_jssp, day_start, planning_end)
df_jssp_curr

,Job,Operation,Machine,Processing Time
0,Job_000,0,M0,29
1,Job_000,1,M1,78
2,Job_000,2,M2,9
3,Job_000,3,M3,36
4,Job_000,4,M4,49
...,...,...,...,...
565,Job_056,5,M9,76
566,Job_056,6,M5,47
567,Job_056,7,M3,52
568,Job_056,8,M4,90


In [28]:
df_times_curr 

,Job,Arrival
0,Job_000,0.00
1,Job_001,83.42
2,Job_002,144.74
3,Job_003,197.03
4,Job_004,233.63
...,...,...
52,Job_052,4008.21
53,Job_053,4014.27
54,Job_054,4015.63
55,Job_055,4134.35


### Scheduling

In [ ]:
starting_time = time.time()

# Scheduling
df_plan = ssv.solve_jssp_flowtime(df_jssp_curr, df_times_currs, solver= "HiGHS", msg=False, timeLimit=max_time, gapRel= 0.10)
df_plan

# Informationen
ending_time = time.time()
solver_duration = ending_time - starting_time
print(f"\nScheduling-Dauer: {int(solver_duration // 60)} Minuten und {(solver_duration % 60):.2f} Sekunden.")
df_plan

In [ ]:
show.plot_gantt_jobs(df_plan)

check.is_machine_conflict_free(df_plan)
check.is_operation_sequence_correct(df_plan)
check.is_job_timing_correct(df_plan)
check.is_start_correct(df_plan)

### Simulation

In [ ]:
simulation = ProductionDaySimulation(df_plan, vc=0.35)
df_execution, df_undone = simulation.run(start_time = day_start, end_time=day_end)

In [ ]:
if not df_execution.empty:
    show.plot_gantt_machines(df_execution, duration_column="Simulated Processing Time")
else:
    print(f"Nothing executed on day {day_numb}")

#### Export

In [ ]:
df_jssp.to_csv("data/05_flowtime_all_jssp.csv", index = False)
df_times.to_csv("data/05_flowtime_all_times.csv", index = False)

In [ ]:
df_plan.to_csv("data/05_flowtime_init_plan.csv", index = False)
df_execution.to_csv("data/05_flowtime_execution.csv", index = False)
df_undone.to_csv("data/05_flowtime_init_undone.csv", index = False)